In [3]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import torch

In [9]:
torch.cuda.get_device_name(torch.device('cuda:0'))

'NVIDIA GeForce RTX 4090'

In [5]:
!git clone https://github.com/huggingface/transformers -b v4.23.1

Cloning into 'transformers'...
remote: Enumerating objects: 157524, done.
remote: Counting objects: 100% (2545/2545), done.
remote: Compressing objects: 100% (1014/1014), done.
remote: Total 157524 (delta 1581), reused 2037 (delta 1286), pack-reused 154979
Receiving objects: 100% (157524/157524), 159.55 MiB | 24.78 MiB/s, done.
Resolving deltas: 100% (117762/117762), done.
Note: switching to 'bd469c40659ce76c81f69c7726759d249b4aef49'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [10]:
%%time

!rm -r output

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=data/train.txt \
    --validation_file=data/train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=50 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/

08/23/2023 17:13:45 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/23/2023 17:13:45 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
h

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("output/").to(device)

In [12]:
input = tokenizer.encode("おはよう", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=8)
tokenizer.batch_decode(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['おはよう</s> おかえり こんばんは こんにちは こんばんは おやすみ こんにちは こんにちは こんばんは こんばんは こんばんは こんばんは こんぴゅーたさま! えへへ...  ちぇっ! もぉ～! もぉ～! もぉ～! もぉ～!...終電逃しちゃったよぉ～...(;;<unk> )ガタガ',
 'おはよう</s> おやすみ こんにちは こんばんわ こんにちわ! こんばんわ! こんばんわっ! ちぇっ! ちぇっ! ちぇっ!  やっ...、こんばんは! えへへ...、こ、これ...、くれる...?...............。...ごめん。私、食べちゃった(汗 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>',
 'おはよう</s> ございまひる! 今日も清々しい一日のはじまりだね、お兄ちゃん!  にこにこ、はりきってあいさつしようね♪ ...つくしごはん、たのしいね♪ ...つくしごはんって、食べておいしいものじゃないの...? </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>',
 'おはよう</s> こんにちは こんばんは ええじいさん! ヨロシク!   ふ～ん、平さん?  ふ～ん、なんか嫌なことあったの? はうぅ～...(しゅん)  なによその、そっけない態度はぁ～...(汗  </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>',
 'おはよう</s> わっしょい がんばれ! みかんたえ者 チャッピー チャッピー朝太郎 チャッピーひな祭り チャッピーおにいちゃん チャッピー月見草 チャッピーのわんぱく大冒険 チャッピーの大冒険2 チャッピーのわんぱく大冒険3 チャッピーのわんぱく大冒険4 チャッピーのだいじょうぶだね チャッピーのだい